In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem

# 1. Setup Molecule
smiles = "C1[C@@H]([C@H]([C@@H]([C@H](O1)O)O)O)O"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# 2. Generate multiple conformers
# numConfs: how many to try. For glucose, 50-100 is usually plenty.
# pruneRmsThresh: removes conformers that are too similar (RMSD < 0.5)
conf_ids = AllChem.EmbedMultipleConfs(mol, numConfs=50, pruneRmsThresh=0.5)

# 3. Optimize all conformers with a Force Field (MMFF)
# This moves each random shape to its nearest local "valley" (minimum)
results = AllChem.MMFFOptimizeMoleculeConfs(mol)

# 4. Extract Energies and Sort
# results is a list of (not_converged, energy)
energies = []
for i, res in enumerate(results):
    energies.append((i, res[1]))

# Sort by energy (lowest first)
energies.sort(key=lambda x: x[1])

print(f"Found {len(energies)} unique conformers.")
print(f"Lowest Energy (MMFF): {energies[0][1]:.4f} kcal/mol")

Found 7 unique conformers.
Lowest Energy (MMFF): 67.4857 kcal/mol


In [4]:
import py3Dmol

view = py3Dmol.view(width=800, height=400)
# Add the first 5 lowest energy conformers
for cid, energy in energies[:5]:
    view.addModel(Chem.MolToMolBlock(mol, confId=cid), 'mol')

view.setStyle({'stick': {}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [5]:
import py3Dmol

# Define grid dimensions (e.g., 2 rows, 3 columns)
rows, cols = 2, 3
view = py3Dmol.view(width=900, height=600, linked=True, viewergrid=(rows, cols))

for i, (cid, energy) in enumerate(energies[:rows*cols]):
    r, c = divmod(i, cols)
    # Convert RDKit conformer to MolBlock
    mb = Chem.MolToMolBlock(mol, confId=cid)
    
    view.addModel(mb, 'mol', viewer=(r, c))
    view.setStyle({'stick': {}}, viewer=(r, c))
    view.addLabel(f"Energy: {energy:.2f}", {'position': {'x':0, 'y':0, 'z':0}, 'backgroundColor': 'white', 'fontColor':'black'}, viewer=(r, c))
    view.zoomTo(viewer=(r, c))

view.render()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.